In [9]:
from transformers import pipeline
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import torch
import re
import json
from spellchecker import SpellChecker  # For spelling and grammar checks

class ResponseAnalyzer:
    def __init__(self):
        # Load a better AI detection model (e.g., OpenAI's GPT-based model or other reliable models)
        self.ai_detector = pipeline("text-classification",
                                  model="roberta-base-openai-detector",  # Replace with a better model
                                  device=0 if torch.cuda.is_available() else -1)
        self.similarity_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
        self.min_word_count = 50  # Minimum word count to consider AI-generated text
        self.spell_checker = SpellChecker()  # For spelling and grammar checks

    def check_ai_generated(self, text):
        """Check if text is AI generated, with adjustments for word count, spelling, and grammar."""
        word_count = len(text.split())

        # Check for poor spelling and grammar
        misspelled = self.spell_checker.unknown(text.split())
        misspelled_count = len(misspelled)

        # Determine if the text is grammatically correct and spelled correctly
        is_grammatically_correct = misspelled_count == 0  # Assume no misspelled words means grammatically correct

        # Conditions for AI-generated text
        is_long_text = word_count > self.min_word_count
        is_well_written = is_grammatically_correct

        # Calculate AI probability based on conditions
        if is_long_text and is_well_written:
            # If all conditions are met, it's most likely AI-generated
            ai_probability = 90.0  # High probability
        elif is_long_text or is_well_written:
            # If at least one condition is met, assign neutral probability
            ai_probability = 50.0  # Neutral probability
        else:
            # If no conditions are met, it's unlikely to be AI-generated
            ai_probability = 0.0  # Low probability

        # Use the AI detector to refine the probability if conditions are met
        if is_long_text or is_well_written:
            result = self.ai_detector(text)
            detector_probability = round(result[0]['score'] * 100, 2)
            # Blend the detector probability with the condition-based probability
            ai_probability = (ai_probability + detector_probability) / 2

        # Ensure AI probability is within bounds
        ai_probability = max(0, min(100, ai_probability))

        return round(ai_probability, 2)

    def check_plagiarism(self, student_answer, reference_text):
        """Check plagiarism against reference text"""
        # Split reference text into chunks
        chunks = [s.strip() for s in reference_text.split('.') if s.strip()]

        # Get embeddings
        student_embedding = self.similarity_model.encode(student_answer, convert_to_tensor=True)
        chunk_embeddings = self.similarity_model.encode(chunks, convert_to_tensor=True)

        # Calculate similarities
        similarities = util.pytorch_cos_sim(student_embedding, chunk_embeddings)
        max_similarity = float(torch.max(similarities)) * 100

        return round(max_similarity, 2)

def analyze_responses(json_file='student_responses.json'):
    # Load the analyzer
    analyzer = ResponseAnalyzer()

    # Read existing responses
    with open(json_file, 'r') as f:
        data = json.load(f)

    # Create lists for DataFrame
    records = []

    # Analyze each response
    for q_data in data['questions']:
        ai_score = analyzer.check_ai_generated(q_data['user_answer'])
        plagiarism_score = analyzer.check_plagiarism(q_data['user_answer'], q_data['correct_answer'])

        record = {
            'question': q_data['question'],
            'user_answer': q_data['user_answer'],
            'correct_answer': q_data['correct_answer'],
            'time_taken': q_data['time_taken'],
            'similarity_score': q_data['similarity'],
            'marks': q_data['marks'],
            'ai_generated_probability': ai_score,
            'plagiarism_probability': plagiarism_score
        }
        records.append(record)

    # Create DataFrame
    df = pd.DataFrame(records)

    # Save to CSV
    csv_filename = 'student_responses_analysis.csv'
    df.to_csv(csv_filename, index=False)

    # Update JSON with new metrics
    for i, q_data in enumerate(data['questions']):
        q_data['ai_generated_probability'] = float(records[i]['ai_generated_probability'])
        q_data['plagiarism_probability'] = float(records[i]['plagiarism_probability'])

    with open(json_file, 'w') as f:
        json.dump(data, f, indent=2)

    # Print summary
    print("\nAnalysis Summary:")
    print(f"Average AI Generation Probability: {df['ai_generated_probability'].mean():.2f}%")
    print(f"Average Plagiarism Probability: {df['plagiarism_probability'].mean():.2f}%")
    print(f"\nDetailed analysis saved to {csv_filename}")

    return df

# Example usage:
if __name__ == "__main__":
    df = analyze_responses()

    # Display sample analysis
    print("\nDetailed Analysis per Question:")
    for idx, row in df.iterrows():
        print(f"\nQuestion {idx + 1}:")
        print(f"AI Generated Probability: {row['ai_generated_probability']}%")
        print(f"Plagiarism Probability: {row['plagiarism_probability']}%")
        print("-" * 50)

Some weights of the model checkpoint at roberta-base-openai-detector were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu



Analysis Summary:
Average AI Generation Probability: 33.10%
Average Plagiarism Probability: 38.38%

Detailed analysis saved to student_responses_analysis.csv

Detailed Analysis per Question:

Question 1:
AI Generated Probability: 0.0%
Plagiarism Probability: 3.11%
--------------------------------------------------

Question 2:
AI Generated Probability: 0.0%
Plagiarism Probability: 5.89%
--------------------------------------------------

Question 3:
AI Generated Probability: 66.94%
Plagiarism Probability: 44.53%
--------------------------------------------------

Question 4:
AI Generated Probability: 65.45%
Plagiarism Probability: 100.0%
--------------------------------------------------
